
# Assignment 2 for Course 1MS041
Make sure you pass the `# ... Test` cells and
 submit your solution notebook in the corresponding assignment on the course website. You can submit multiple times before the deadline and your highest score will be used.

---
## Assignment 2, PROBLEM 1
Maximum Points = 8


A courier company operates a fleet of delivery trucks that make deliveries to different parts of the city. The trucks are equipped with GPS tracking devices that record the location of each truck at regular intervals. The locations are divided into three regions: downtown, the suburbs, and the countryside. The following table shows the probabilities of a truck transitioning between these regions at each time step:

| Current region | Probability of transitioning to downtown | Probability of transitioning to the suburbs | Probability of transitioning to the countryside |
|----------------|--------------------------------------------|-----------------------------------------------|------------------------------------------------|
| Downtown       | 0.3                                      | 0.4                                           | 0.3                                            |
| Suburbs        | 0.2                                      | 0.5                                           | 0.3                                            |
| Countryside    | 0.4                                      | 0.3                                           | 0.3                                            |

1. If a truck is currently in the suburbs, what is the probability that it will be in the downtown region after two time steps? [1.5p]
2. If a truck is currently in the suburbs, what is the probability that it will be in the downtown region **the first time** after two time steps? [1.5p]
3. Is this Markov chain irreducible? [1.5p]
4. What is the stationary distribution? [1.5p]
5. Advanced question: What is the expected number of steps until the first time one enters the downtown region having started in the suburbs region. Hint: to get within 1 decimal point, it is enough to compute the probabilities for hitting times below 30. [2p]



In [5]:
# Part 1

# Fill in the answer to part 1 below as a decimal number
# Can calculate this by hand but can also solve it in code using a transition matrix:
import numpy as np

transition_matrix = np.array([
    [0.3, 0.4, 0.3],  # From Downtown to [Downtown, Suburbs, Countryside]
    [0.2, 0.5, 0.3],  # From Suburbs to [Downtown, Suburbs, Countryside]
    [0.4, 0.3, 0.3]   # From Countryside to [Downtown, Suburbs, Countryside]
])

# Now enetering our initial conditions for part 1:
initial_state = np.array([0, 1, 0])  # Truck starts in Suburbs

# Calculate the state probability vector after two time steps
# P(t+2) = initial_state * (transition_matrix)^2
two_step_transition_matrix = np.linalg.matrix_power(transition_matrix, 2)
two_step_probabilities = initial_state @ two_step_transition_matrix

# Probability that the truck will be in Downtown after two steps
problem1_p1 = two_step_probabilities[0]
problem1_p1

0.28

In [3]:
# Part 2

# Fill in the answer to part 2 below as a decimal number

# P(suburbs -> suburbs) + P(suburbs -> countryside)
prob_not_downtown_first_step = transition_matrix[1, 1] + transition_matrix[1, 2]
prob_suburbs_to_downtown = transition_matrix[1, 0]  # Probability from suburbs to downtown
prob_countryside_to_downtown = transition_matrix[2, 0]  # Probability from countryside to downtown

# Total probability: (Probability of not downtown in step 1) * (probability of moving to downtown in step 2)
prob_first_time_downtown_after_two_steps = (
    transition_matrix[1, 1] * prob_suburbs_to_downtown +
    transition_matrix[1, 2] * prob_countryside_to_downtown
)

problem1_p2 = prob_first_time_downtown_after_two_steps
problem1_p2

0.22

In [4]:
# Part 3

# Fill in the answer to part 3 below as a boolean
problem1_irreducible = True

In [17]:
eigenvectors

array([[-4.93651112e-01, -8.16496581e-01,  7.07106781e-01],
       [-7.02503505e-01,  4.08248290e-01, -7.07106781e-01],
       [-5.12637693e-01,  4.08248290e-01,  6.76450183e-16]])

In [20]:
eigenvectors[:,1].flatten()

array([-0.81649658,  0.40824829,  0.40824829])

In [10]:
# Part 4

# Fill in the answer to part 4 below
# the answer should be a numpy array of length 3
# make sure that the entries sums to 1!
from scipy.linalg import eig

# Compute the eigenvalues and eigenvectors of the transition matrix
eigenvalues, eigenvectors = eig(transition_matrix.T)

# Find the eigenvector associated with eigenvalue 1 (stationary distribution)
# Identify the index of the eigenvalue closest to 1
index_of_1 = np.isclose(eigenvalues, 1)
stationary_vector = np.real(eigenvectors[:, index_of_1]).flatten()

# Normalize to make it a probability distribution (sum to 1)
stationary_distribution = stationary_vector / np.sum(stationary_vector)
stationary_distribution_rounded = np.round(stationary_distribution, 2)

problem1_stationary = stationary_distribution_rounded
problem1_stationary

array([0.29, 0.41, 0.3 ])

In [6]:
index_of_1

array([ True, False, False])

In [23]:
# Part 5

# Fill in the answer to part 5 below
# That is, the expected number of steps as a decimal number
# Initialize parameters
def calculate_hitting_time_distribution(max_steps=30):
    # Initialize transition matrix P
    P = transition_matrix
    # Initialize array to store probability of hitting downtown at exactly step k
    hitting_probs = np.zeros(max_steps + 1)
    
    # Initial state distribution (starting from suburbs)
    current_dist = np.zeros(3)
    current_dist[1] = 1.0
    
    # For first step, it's simply the transition probability
    hitting_probs[1] = P[1,0]

    # Modified transition matrix that removes transitions from downtown
    P_modified = P.copy()
    P_modified[0,:] = 0  # Once we hit downtown, we stop
    
    # Calculate probabilities for each step
    for k in range(2, max_steps + 1):
        # Update distribution but exclude paths that already hit downtown
        current_dist = current_dist @ P_modified
        # Probability of hitting downtown at exactly step k
        hitting_probs[k] = current_dist[1] * P[1,0] + current_dist[2] * P[2,0]
    
    return hitting_probs

# Calculate expected value
def calculate_expected_steps():
    hitting_probs = calculate_hitting_time_distribution()
    steps = np.arange(len(hitting_probs))
    expected_steps = np.sum(steps * hitting_probs)
    
    # Calculate cumulative probability to check if we're missing significant mass
    total_prob = np.sum(hitting_probs)
    print(hitting_probs)
    print(expected_steps)
    
    print(f"Expected number of steps: {expected_steps:.1f}")
    print(f"Total probability captured: {total_prob:.3f}")
    
    return expected_steps

# Run the calculation
problem1_ET = calculate_expected_steps()
round(problem1_ET,1)

[0.00000000e+00 2.00000000e-01 2.20000000e-01 1.64000000e-01
 1.18000000e-01 8.45600000e-02 6.05680000e-02 4.33808000e-02
 3.10705600e-02 2.22536000e-02 1.59386464e-02 1.14157011e-02
 8.17624211e-03 5.85605162e-03 4.19426677e-03 3.00405032e-03
 2.15158425e-03 1.54102438e-03 1.10372445e-03 7.90518097e-04
 5.66191010e-04 4.05521722e-04 2.90445917e-04 2.08025431e-04
 1.48993589e-04 1.06713346e-04 7.64310612e-05 5.47420482e-05
 3.92077749e-05 2.80816970e-05 2.01128911e-05]
3.8444520365146007
Expected number of steps: 3.8
Total probability captured: 1.000


3.8

In [22]:
hitting_probs

NameError: name 'hitting_probs' is not defined

---
## Assignment 2, PROBLEM 2
Maximum Points = 4


Use the **Multi-dimensional Constrained Optimisation** example (in `07-Optimization.ipynb`) to numerically find the MLe for the mean and variance parameter based on `normallySimulatedDataSamples`, an array obtained by a specific simulation of $30$ IID samples from the $Normal(10,2)$ random variable.

Recall that $Normal(\mu, \sigma^2)$ RV has the probability density function given by:

$$
f(x ;\mu, \sigma) = \displaystyle\frac{1}{\sigma\sqrt{2\pi}}\exp\left(\frac{-1}{2\sigma^2}(x-\mu)^2\right)
$$

The two parameters, $\mu \in \mathbb{R} := (-\infty,\infty)$ and $\sigma \in (0,\infty)$, are sometimes referred to as the location and scale parameters.

You know that the log likelihood function for $n$ IID samples from a Normal RV with parameters $\mu$ and $\sigma$ simply follows from $\sum_{i=1}^n \log(f(x_i; \mu,\sigma))$, based on the IID assumption. 

NOTE: When setting bounding boxes for $\mu$ and $\sigma$ try to start with some guesses like $[-20,20]$ and $[0.1,5.0]$ and make it larger if the solution is at the boundary. Making the left bounding-point for $\sigma$ too close to $0.0$ will cause division by zero Warnings. Other numerical instabilities can happen in such iterative numerical solutions to the MLe. You need to be patient and learn by trial-and-error. You will see the mathematical theory in more details in a future course in scientific computing/optimisation. So don't worry too much now except learning to use it for our problems.  

In [8]:

import numpy as np
from scipy import optimize
# do NOT change the next three lines
np.random.seed(123456) # set seed
# simulate 30 IID samples drawn from Normal(10,2)RV
normallySimulatedDataSamples = np.random.normal(10,2,30) 

# define the negative log likelihoo function you want to minimise by editing XXX
def negLogLklOfIIDNormalSamples(parameters):
    '''return the -log(likelihood) of normallySimulatedDataSamples with mean and var parameters'''
    mu_param=parameters[0]
    sigma_param=parameters[1]
     # Compute negative log likelihood for normal distribution
    n = len(normallySimulatedDataSamples)
    log_likelihood = -n * np.log(sigma_param) - n/2 * np.log(2*np.pi) - \
                    np.sum((normallySimulatedDataSamples - mu_param)**2)/(2*sigma_param**2)
    
    return -log_likelihood  # Return negative since we want to minimize
# you should only change XXX below and not anything else
parameter_bounding_box=((-20, 20), (0.1, 10.0)) # specify the constraints for each parameter - some guess work...
initial_arguments = np.array([0.0, 1.0]) # point in 2D to initialise the minimize algorithm
result_problem2_opt = optimize.minimize(negLogLklOfIIDNormalSamples, initial_arguments, bounds=parameter_bounding_box, method='L-BFGS-B') 
# call the minimize method above finally! you need to play a bit to get initial conditions and bounding box ok
result_problem2_opt


  message: CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
  success: True
   status: 0
      fun: 58.63138728318369
        x: [ 9.269e+00  1.708e+00]
      nit: 9
      jac: [-8.313e-05  1.400e-04]
     nfev: 69
     njev: 23
 hess_inv: <2x2 LbfgsInvHessProduct with dtype=float64>

---
## Assignment 2, PROBLEM 3
Maximum Points = 4



Derive the maximum likelihood estimate for $n$ IID samples from a random variable with the following probability density function:
$$
f(x; \lambda) = \frac{1}{24} \lambda^5 x^4 \exp(-\lambda x), \qquad \text{ where, } \lambda>0, x > 0
$$

You can solve the MLe by hand (using pencil paper or using key-strokes). Present your solution as the return value of a function called `def MLeForAssignment2Problem3(x)`, where `x` is a list of $n$ input data points.

In [9]:

# do not change the name of the function, just replace XXX with the appropriate expressions for the MLe
def MLeForAssignment2Problem3(x):
    '''
    Calculates the Maximum Likelihood Estimate for lambda parameter
    in the density function f(x;λ)=(1/24λ⁵)x⁴exp(-λx)
    
    Args:
        x: list or array of n IID samples from the distribution
        
    Returns:
        float: MLE estimate for lambda parameter
    '''
    n = len(x)
    sum_x = np.sum(x)
    lambda_mle = 5 * n / sum_x
    
    return lambda_mle
samples = [1, 2, 3, 4, 5]
lambda_est = MLeForAssignment2Problem3(samples)
lambda_est

1.6666666666666667

---
## Assignment 2, PROBLEM 4
Maximum Points = 8


## Random variable generation and transformation

The purpose of this problem is to show that you can implement your own sampler, this will be built in the following three steps:

1. [2p] Implement a Linear Congruential Generator where you tested out a good combination (a large $M$ with $a,b$ satisfying the Hull-Dobell (Thm 6.8)) of parameters. Follow the instructions in the code block.
2. [2p] Using a generator construct random numbers from the uniform $[0,1]$ distribution.
3. [4p] Using a uniform $[0,1]$ random generator, generate samples from 

$$p_0(x) = \frac{\pi}{2}|\sin(2\pi x)|, \quad x \in [0,1] \enspace .$$

Using the **Accept-Reject** sampler (**Algorithm 1** in TFDS notes) with sampling density given by the uniform $[0,1]$ distribution.

In [10]:

def problem4_LCG(size=None, seed = 0):
    """
    A linear congruential generator that generates pseudo random numbers according to size.
    Using parameters that satisfy Hull-Dobell theorem for maximum period.
    """
    if size is None:
        return []
    
    # Choose parameters satisfying Hull-Dobell theorem:
    # M should be large prime
    M = 2**31 - 1  # Mersenne prime, commonly used
    # a should be primitive root modulo M
    a = 48271      # Known primitive root for this M
    # b = 0 is fine since M is prime
    b = 0
    
    numbers = []
    current = seed
    
    for _ in range(size):
        current = (a * current + b) % M
        numbers.append(current)
    
    return numbers

In [11]:

def problem4_uniform(generator=None, period = 1, size=None, seed=0):
    """
    Takes a generator and produces samples from the uniform [0,1] distribution.
    """
    if size is None or generator is None:
        return []
    
    # Get raw numbers from the generator
    raw_numbers = generator(size=size, seed=seed)
    
    # Convert to uniform [0,1] by dividing by period
    uniform_numbers = [x / period for x in raw_numbers]
    
    return uniform_numbers

In [12]:

def problem4_accept_reject(uniformGenerator=None, n_iterations=None, seed=0):
    """
    Accept-reject sampler for p0(x) = (pi/2)|sin(2πx)| using uniform as proposal.
    """
    if n_iterations is None or uniformGenerator is None:
        return []
    
    accepted_samples = []
    M = np.pi/2  # Maximum value of the target density
    
    # Generate uniform samples for both x and acceptance criterion
    uniform_samples = uniformGenerator(size=2*n_iterations, seed=seed)
    
    # Process pairs of uniform samples
    for i in range(0, len(uniform_samples), 2):
        if i + 1 >= len(uniform_samples):
            break
            
        x = uniform_samples[i]
        u = uniform_samples[i + 1]
        
        # Target density
        target_density = (np.pi/2) * abs(np.sin(2 * np.pi * x))
        
        # Accept-reject criterion
        if u * M <= target_density:
            accepted_samples.append(x)
    
    return accepted_samples

---
#### Local Test for Assignment 2, PROBLEM 4
Evaluate cell below to make sure your answer is valid.                             You **should not** modify anything in the cell below when evaluating it to do a local test of                             your solution.
You may need to include and evaluate code snippets from lecture notebooks in cells above to make the local test work correctly sometimes (see error messages for clues). This is meant to help you become efficient at recalling materials covered in lectures that relate to this problem. Such local tests will generally not be available in the exam.

In [17]:

# If you managed to solve all three parts you can test the following code to see if it runs
# you have to change the period to match your LCG though, this is marked as XXX.
# It is a very good idea to check these things using the histogram function in sagemath
# try with a larger number of samples, up to 10000 should run

#print("LCG output: %s" % problem4_LCG(size=1000, seed = 1))

period = 2**31 - 1

print("Uniform sampler %s" % problem4_uniform(generator=problem4_LCG, period = period, size=10, seed=1))

uniform_sampler = lambda size,seed: problem4_uniform(generator=problem4_LCG, period = period, size=size, seed=seed)

#print("Accept-Reject sampler %s" % problem4_accept_reject(uniformGenerator = uniform_sampler,n_iterations=20,seed=1))
#from sage.plot.histogram import Histogram
#g = Histogram(problem4_LCG(size=1000, seed = 1), {}); g

Uniform sampler [2.2477936010098986e-05, 0.08503244914348818, 0.6013526053174179, 0.8916112770753034, 0.9679557019695433, 0.18968977182623453, 0.514975824167475, 0.39800838818680884, 0.26290616545030204, 0.7435124515292758]


In [13]:

# If however you did not manage to implement either part 1 or part 2 but still want to check part 3, you can run the code below

def testUniformGenerator(size,seed):
    import random
    random.seed(seed)
    
    return [random.uniform(0,1) for s in range(size)]

print("Accept-Reject sampler %s" % problem4_accept_reject(uniformGenerator=testUniformGenerator, n_iterations=20, seed=1))

Accept-Reject sampler [0.763774618976614, 0.651592972722763, 0.0938595867742349, 0.8357651039198697, 0.762280082457942, 0.22876222127045265, 0.9014274576114836, 0.21659939713061338, 0.23308445025757263, 0.2187810373376886, 0.28978161459048557, 0.8375779756625729, 0.6422943629324456]
